In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.4 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import pandas as pd
import re

pages_to_extract = [1, 2]
all_data_rows = []

date_pattern = re.compile(r'^(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\s+\d{1,2}\s+[A-Za-z]+', re.IGNORECASE)

expected_cols = 6

with pdfplumber.open('/content/Trading_Schedule.pdf') as f:
    for page_num in pages_to_extract:
        page = f.pages[page_num]
        tables = page.extract_tables()

        if tables:
            for table in tables:
                for row in table:
                    if row and row[0] and date_pattern.match(row[0]):

                        if len(row) < expected_cols:
                            row.extend([''] * (expected_cols - len(row)))

                        if page_num > 1:

                            date = row[0]
                            col1 = row[1]
                            col2 = row[2]
                            col6 = row[3]


                            row = [
                                date,
                                '',
                                '',
                                col1,
                                col2,
                                col6
                            ]

                        all_data_rows.append(row)

df = pd.DataFrame(all_data_rows, columns=['Date', 'Col2', 'Col3', 'Col4', 'Col5', 'Col6'])
df.to_csv('x.csv', index=False)
print("saved to x.csv")


saved to x.csv


In [ ]:
import pandas as pd
import re
from datetime import datetime

def parse_date_dynamic(date_str):
    try:
        return pd.to_datetime(date_str, format="%A %d %B %Y").strftime("%d/%m/%Y")
    except:
        try:
            match = re.match(r"(\w+ \d{1,2} \w+)", date_str)
            if match:
                date_without_year = match.group(1)
                for year in range(2024, 2030):
                    full_date = f"{date_without_year} {year}"
                    try:
                        parsed = pd.to_datetime(full_date, format="%A %d %B %Y")

                        if parsed.strftime("%A %d %B") == date_without_year:
                            return parsed.strftime("%d/%m/%Y")
                    except:
                        continue
        except:
            return None
    return None

df_full = pd.read_csv("x.csv", header=None)

df_full = df_full.drop([1, 2], axis=1)

df_trimmed = df_full.iloc[4:].reset_index(drop=True)
df_trimmed.columns = ['Date', 'Occasion', 'Status', 'Additional Information']

valid_rows = []
for _, row in df_trimmed.iterrows():
    if pd.isna(row['Date']) or str(row['Date']).strip() == "":
        break
    valid_rows.append(row)

df_cleaned = pd.DataFrame(valid_rows).reset_index(drop=True)

df_cleaned['Date'] = df_cleaned['Date'].apply(parse_date_dynamic)

print(df_cleaned.head())

df_cleaned.to_csv("cleaned_data.csv", index=False)

         Date                        Occasion  Status  \
0  17/02/2025      Presidents Day\nUS Holiday       A   
1  28/03/2025                 Hari Raya Puasa      TD   
2  17/04/2025                 Easter Thursday      TD   
3  18/04/2025  Good Friday\nUK and US Holiday  Closed   
4  21/04/2025       Easter Monday\nUK Holiday      TD   

                              Additional Information  
0  For further details, please refer to Trading S...  
1  Singapore Markers & Transitioned Oil TAPS 04:2...  
2  Singapore Markers & Transitioned Oil TAPS 05:2...  
3                                                NaN  
4  IFEU Agricultural Commodities Closed\nUK Natur...  


In [ ]:
import pdfplumber
import json
import re

def extract_trading_schedule_key(pdf_path):
    # Initialize the dictionary to store the extracted data
    trading_schedule = {}

    # Define the keys to extract
    keys = ['A', 'B', 'C', 'D']

    try:
        # Open the PDF file
        with pdfplumber.open(pdf_path) as pdf:
            print(f"PDF loaded successfully. Total pages: {len(pdf.pages)}")

            # Target pages 4 and 5 where TRADING SCHEDULE KEY is located
            target_pages = [3, 4]  # Page 4 (index 3) and Page 5 (index 4)
            full_text = ""

            # Extract text from target pages
            for page_num in target_pages:
                page = pdf.pages[page_num]
                text = page.extract_text()
                if text:
                    full_text += text + "\n"
                    print(f"\n--- Page {page_num + 1} Text Preview ---\n{text[:200]}...\n")
                else:
                    print(f"Page {page_num + 1}: No text extracted.")

            # Split the combined text into lines
            lines = full_text.split('\n')

            current_key = None
            current_content = []
            in_schedule_key = False

            # Regex to match keys at the start of a line
            key_pattern = re.compile(r'^\s*([ABCD])(?:\s+|$)', re.MULTILINE)

            for i, line in enumerate(lines):
                # Skip empty lines or irrelevant headers/footers
                if not line.strip() or line.startswith('IFEU Trading Schedule') or line.startswith('© 2025') or line.startswith('Q2005'):
                    continue

                # Detect start of TRADING SCHEDULE KEY section
                if 'TRADING SCHEDULE KEY' in line:
                    in_schedule_key = True
                    print("Found TRADING SCHEDULE KEY section")
                    continue

                # Only process lines within the TRADING SCHEDULE KEY section
                if in_schedule_key:
                    # Check if the line starts with a key (A, B, C, D)
                    match = key_pattern.match(line)
                    if match:
                        key = match.group(1)
                        # If we were collecting content for a previous key, save it
                        if current_key:
                            trading_schedule[current_key] = '\n'.join(current_content).strip()
                            print(f"Extracted key {current_key} with content: {trading_schedule[current_key][:100]}...")
                        # Start collecting for the new key
                        current_key = key
                        current_content = [line[len(key):].strip()]  # Include the rest of the line after the key
                        print(f"Found key {current_key} at line: {line}")
                    elif current_key:
                        # Add line to the current key's content
                        current_content.append(line.strip())

            # Save the last key's content
            if current_key and current_content:
                trading_schedule[current_key] = '\n'.join(current_content).strip()
                print(f"Extracted key {current_key} with content: {trading_schedule[current_key][:100]}...")

            if not trading_schedule:
                print("No keys (A, B, C, D) found in the TRADING SCHEDULE KEY section.")
            else:
                print(f"Extracted keys: {list(trading_schedule.keys())}")

    except FileNotFoundError:
        print(f"Error: The file '{pdf_path}' was not found. Ensure it is in the same directory as the script.")
    except Exception as e:
        print(f"Error occurred: {str(e)}")

    return trading_schedule

def save_to_json(data, output_path):
    # Save the extracted data to a JSON file
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)
    print(f"Data saved to {output_path}")

def main():
    pdf_path = "Trading_Schedule.pdf"
    output_path = "trading_schedule_key.json"

    # Extract the trading schedule key data
    trading_schedule = extract_trading_schedule_key(pdf_path)

    # Save the extracted data to a JSON file if not empty
    if trading_schedule:
        save_to_json(trading_schedule, output_path)
    else:
        print("No data extracted to save.")

if __name__ == "__main__":
    main()

PDF loaded successfully. Total pages: 10

--- Page 4 Text Preview ---
TRADING SCHEDULE KEY
TD
Regular Trading Hours and Settlement Times
(Trading Day)
A Trading Hours
Regular Trading Hours EXCEPT for the following contracts:
White Sugar Futures & Options early close at ...


--- Page 5 Text Preview ---
C Trading Hours
Early close at 18:30 UK (13:30 ET)
Brent, WTI, Midland WTI AGC and Permian WTI Storage Futures & Options
Low Sulphur Gasoil Futures & Options
Dubai Crude Futures & Options
Heating Oil,...

Found TRADING SCHEDULE KEY section
Found key A at line: A Trading Hours
Extracted key A with content: Trading Hours
Regular Trading Hours EXCEPT for the following contracts:
White Sugar Futures & Option...
Found key B at line: B Trading Hours
Extracted key B with content: Trading Hours
Early close at 20:00 UK (15:00 ET)
Brent, WTI, Midland WTI AGC and Permian WTI Storage...
Found key C at line: C Trading Hours
Extracted key C with content: Trading Hours
Early close at 18:30 UK (13:30 ET

In [ ]:
import pandas as pd
from datetime import datetime
import json
import re

pd.set_option('display.max_colwidth', None)

df = pd.read_csv("cleaned_data.csv")
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y", errors='coerce')
df = df.dropna(subset=['Date'])

with open("/content/trading_schedule_key.json", "r") as f:
    status_info = json.load(f)

def parse_sections(text):
    commodities = []
    times = []
    current_section = None

    # Split text into lines and process
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        if line in ["Settlement Times", "Marker Times"]:
            current_section = line
            continue
        if current_section == "Settlement Times":
            # Match standard settlement time pattern
            if re.match(r"Designated settlement period \d{2}:\d{2}\s*-\s*\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*–\s*\d{2}:\d{2}\s*ET\)", line):
                time = re.search(r"(\d{2}:\d{2}\s*-\s*\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*–\s*\d{2}:\d{2}\s*ET\))", line).group(1)
                times.append(time)
                commodities.append("")  # Placeholder for commodity
            # Match White Sugar settlement time
            elif re.match(r"White Sugar Futures & Options Designated settlement period \d{2}:\d{2}-\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*–\s*\d{2}:\d{2}\s*ET\)", line):
                time = re.search(r"(\d{2}:\d{2}-\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*–\s*\d{2}:\d{2}\s*ET\))", line).group(1)
                times.append(time)
                commodities.append("White Sugar Futures & Options")
            # Match commodity lines
            elif not line.startswith("**") and commodities and commodities[-1] == "":
                commodities[-1] = line  # Update placeholder with commodity
        elif current_section == "Marker Times":
            # Match Low Sulphur Gasoil marker time
            if re.match(r"Low Sulphur Gasoil Futures US Minute Marker calculated at \d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*ET\)", line):
                time = re.search(r"(\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*ET\))", line).group(1)
                times.append(time)
                commodities.append("Low Sulphur Gasoil Futures")

    # Split grouped commodities
    final_commodities = []
    final_times = []
    for comm, time in zip(commodities, times):
        if comm:  # Skip empty commodities
            # Split commodities like "Brent, WTI, Midland WTI AGC, Dubai Crude & Permian WTI Storage Futures & Options"
            comm_list = [c.strip() for c in comm.replace(" & ", ", ").split(", ")]
            for c in comm_list:
                final_commodities.append(c)
                final_times.append(time)

    # Create DataFrame
    return pd.DataFrame({
        "Commodity": final_commodities,
        "Time": final_times
    })

input_date_str = input("Enter a date (dd/mm/yyyy): ")

try:
    input_date = datetime.strptime(input_date_str, "%d/%m/%Y")
    df['Date_Diff'] = df['Date'].apply(lambda x: abs((x - input_date).days))
    closest_row = df.loc[df['Date_Diff'].idxmin()].drop('Date_Diff')

    print("\nClosest Matching Date Entry:\n")
    print(closest_row)

    status_value = str(closest_row.get('Status', '')).strip().upper()
    if status_value in status_info:
        print(f"\nSettlement Times for status '{status_value}':\n")
        print(parse_sections(status_info[status_value]).to_string(index=False))
    else:
        print("\nNo mapped info found for status:", status_value)

except ValueError:
    print("Invalid date format. Please enter the date in dd/mm/yyyy format.")

Enter a date (dd/mm/yyyy): 11/11/2025

Closest Matching Date Entry:

Date                                                                          2025-11-27 00:00:00
Occasion                                                                 Thanksgiving\nUS Holiday
Status                                                                                          C
Additional Information    For further details, please refer to Trading Schedule Key\non pages 4-5
Name: 13, dtype: object

Settlement Times for status 'C':

Commodity                   Time                              
                      Brent 17:58 -18:00 UK (12:58 – 13:00 ET)
                        WTI 17:58 -18:00 UK (12:58 – 13:00 ET)
            Midland WTI AGC 17:58 -18:00 UK (12:58 – 13:00 ET)
                Dubai Crude 17:58 -18:00 UK (12:58 – 13:00 ET)
Permian WTI Storage Futures 17:58 -18:00 UK (12:58 – 13:00 ET)
                    Options 17:58 -18:00 UK (12:58 – 13:00 ET)
        White Sugar Futures  16:53-16:5

In [ ]:
import pandas as pd
from datetime import datetime
import json
import re

# Set pandas display options for better formatting
pd.set_option('display.max_colwidth', None)
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.expand_frame_repr', False)

# Read the CSV and clean the data
df = pd.read_csv("cleaned_data.csv")
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y", errors='coerce')
df = df.dropna(subset=['Date'])

# Load the trading schedule JSON
with open("trading_schedule_key.json", "r") as f:
    status_info = json.load(f)

def parse_sections(text):
    commodities = []
    times = []
    current_section = None

    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        if line in ["Settlement Times", "Marker Times"]:
            current_section = line
            continue
        if current_section == "Settlement Times":
            if re.match(r"Designated settlement period \d{2}:\d{2}\s*-\s*\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*–\s*\d{2}:\d{2}\s*ET\)", line):
                time = re.search(r"(\d{2}:\d{2}\s*-\s*\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*–\s*\d{2}:\d{2}\s*ET\))", line).group(1)
                times.append(time)
                commodities.append("")
            elif re.match(r"White Sugar Futures & Options Designated settlement period \d{2}:\d{2}-\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*–\s*\d{2}:\d{2}\s*ET\)", line):
                time = re.search(r"(\d{2}:\d{2}-\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*–\s*\d{2}:\d{2}\s*ET\))", line).group(1)
                times.append(time)
                commodities.append("White Sugar Futures & Options")
            elif not line.startswith("**") and commodities and commodities[-1] == "":
                commodities[-1] = line
        elif current_section == "Marker Times":
            if re.match(r"Low Sulphur Gasoil Futures US Minute Marker calculated at \d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*ET\)", line):
                time = re.search(r"(\d{2}:\d{2}\s*UK\s*\(\d{2}:\d{2}\s*ET\))", line).group(1)
                times.append(time)
                commodities.append("Low Sulphur Gasoil Futures")

    final_commodities = []
    final_times = []
    for comm, time in zip(commodities, times):
        if comm:
            comm_list = [c.strip() for c in comm.replace(" & ", ", ").split(", ")]
            for c in comm_list:
                final_commodities.append(c)
                final_times.append(time)

    return pd.DataFrame({
        "Commodity": final_commodities,
        "Settlement Time": final_times
    })

def format_table(df, col_widths):
    """Custom function to format table with left-aligned content."""
    columns = df.columns
    data = df.values
    col_widths = col_widths or {col: max(len(str(col)), max(len(str(x)) for x in df[col])) for col in columns}

    # Format header (left-aligned)
    header = "  ".join(f"{col:<{col_widths[col]}}" for col in columns)
    # Format rows (left-aligned)
    rows = [
        "  ".join(f"{str(val):<{col_widths[col]}}" for val, col in zip(row, columns))
        for row in data
    ]
    # Combine header and rows
    return "\n".join([header] + rows)

# Get user input
input_date_str = input("Enter a date (dd/mm/yyyy): ")

try:
    input_date = datetime.strptime(input_date_str, "%d/%m/%Y")
    df['Date_Diff'] = df['Date'].apply(lambda x: abs((x - input_date).days))
    closest_row = df.loc[df['Date_Diff'].idxmin()].copy()
    closest_row = closest_row.drop('Date_Diff')

    # Clean up newline characters in the row
    for col in closest_row.index:
        if isinstance(closest_row[col], str):
            closest_row[col] = closest_row[col].replace('\n', ' ').strip()

    # Format the output
    print("\nClosest Matching Date Entry:\n")
    print("Date:               ", closest_row['Date'].strftime("%d/%m/%Y"))
    print("Occasion:           ", closest_row.get('Occasion', 'N/A'))
    print("Status:             ", closest_row.get('Status', 'N/A'))
    print("Additional Info:    ", closest_row.get('Additional Information', 'N/A'))

    # Process settlement times
    status_value = str(closest_row.get('Status', '')).strip().upper()
    if status_value in status_info:
        print(f"\nSettlement Times for Status '{status_value}':\n")
        settlement_df = parse_sections(status_info[status_value])
        print(format_table(settlement_df, col_widths={'Commodity': 30, 'Settlement Time': 30}))
    else:
        print("\nNo mapped info found for status:", status_value)

except ValueError as e:
    print(f"Invalid date format. Please enter the date in dd/mm/yyyy format (e.g., 01/02/2025). Error: {str(e)}")

Enter a date (dd/mm/yyyy): 11/11/2025

Closest Matching Date Entry:

Date:                27/11/2025
Occasion:            Thanksgiving US Holiday
Status:              C
Additional Info:     For further details, please refer to Trading Schedule Key on pages 4-5

Settlement Times for Status 'C':

Commodity                       Settlement Time               
Brent                           17:58 -18:00 UK (12:58 – 13:00 ET)
WTI                             17:58 -18:00 UK (12:58 – 13:00 ET)
Midland WTI AGC                 17:58 -18:00 UK (12:58 – 13:00 ET)
Dubai Crude                     17:58 -18:00 UK (12:58 – 13:00 ET)
Permian WTI Storage Futures     17:58 -18:00 UK (12:58 – 13:00 ET)
Options                         17:58 -18:00 UK (12:58 – 13:00 ET)
White Sugar Futures             16:53-16:55 UK (11:53 – 11:55 ET)
Options                         16:53-16:55 UK (11:53 – 11:55 ET)
Low Sulphur Gasoil Futures      18:00 UK (13:00 ET)           
